George Mason University, Fairfax, VA 22030

Department of Data Analytics Engineering

Course : DAEN 690

Team name: Team Raiders

Project: Transport Decision Making

Instructor : Dr Issac K Gang

Team Members:

Rakesh Subramani Kaleeshwaran
Bhavana Emmadi
Siri Chandana Keerthipati
Aishwarya Sharma

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import bs4 as bs
from datetime import timedelta
import urllib.request
import pandas as pd
import numpy as np
import kmodes
from kmodes.kmodes import KModes
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

import matplotlib
# import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
#from pyemma import msm # not available on Kaggle Kernel
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

Loading the necessary library and the dataset that will be used to derive the final variable as it is shown
above and below.

In [293]:
data = pd.read_csv("/Volumes/Personal/DAEN 690 - Project/data_model.csv")
data = data.drop(['Unnamed: 0'], axis =1)

Dropping the unnecessary columns which are ID's and names that doesn't give any input to our model.

Trying to reduce the values in FinalIncident type by grouping the similar categories together and low number of categories to 'other' category.

In [294]:
data['FinalIncidentType'][(data['FinalIncidentType'] == 'ACCIF') | (data['FinalIncidentType'] == 'ACCITF') | (data['FinalIncidentType'] == 'ACCIMF') | (data['FinalIncidentType'] == 'ACCIHF') | (data['FinalIncidentType'] == 'ACCIBF')] = "ACCI"
data['FinalIncidentType'][data['FinalIncidentType'].isin(data['FinalIncidentType'].value_counts()[data['FinalIncidentType'].value_counts() < 100].index)] = "OTHERS"
data['FinalIncidentType'][(data['FinalIncidentType'] == 'STABF') | (data['FinalIncidentType'] == 'SHOTF')] = "ASSAULT"
# data['FinalIncidentType'][(data['FinalIncidentType'] == 'AMED') | (data['FinalIncidentType'] == 'PSERVF')] = "FIRE"

data['FinalIncidentType'].value_counts()

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/2871453545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FinalIncidentType'][(data['FinalIncidentType'] == 'ACCIF') | (data['FinalIncidentType'] == 'ACCITF') | (data['FinalIncidentType'] == 'ACCIMF') | (data['FinalIncidentType'] == 'ACCIHF') | (data['FinalIncidentType'] == 'ACCIBF')] = "ACCI"
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/2871453545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['FinalIncidentType'][data['FinalIncidentType'].isin(data['FinalIncidentType'].value_counts()[data['FinalIncidentType'].value

ALS        69185
MEDICAL    35413
ACCI        9446
ODF         5152
CPRF        3027
OTHERS       422
SUIAF        341
AMED         280
ASSAULT      275
PSERVF       242
Name: FinalIncidentType, dtype: int64

In [295]:
data.columns

Index(['ShiftDay', 'TourOfShift', 'IncidentFirstDue', 'InitialIncidentType',
       'DispatchedIncidentType', 'ArrivedIncidentType', 'FinalIncidentType',
       'FinalIncidentTypeDescription', 'ResponseUnitID', 'UnitStation',
       'UnitType', 'UnitArrivalOrder', 'PatientID', 'TransportUnitID',
       'TransportLOC', 'TransportHospitalCode', 'ReasonForChoosingHospital',
       'PatientAge', 'Total_Number_of_procedures_performed',
       'Procedure_given_ALS?', 'Total_Number_of_medications_given',
       'Medication_given_ALS?', 'Number_of_units_responded',
       'Number_of_units_transported', 'Travel_time_taken',
       'Time_taken_to_be_atscene', 'Total_time_from_report_hospital',
       'clean_primary_impression', 'Number of providers',
       'SecondaryImpression', 'Unitotheractions_taken',
       'Unitprimaryactions_taken', 'ProcedurePerformed', 'MedicationGiven',
       'Facility Name', 'Facility_Type_Of_Facility',
       'Facility_Hospital_Designation_List', 'Distance'],
      

To correct the categories in the medications_given_ALS and Procedure_given_ALS to a proper format.
That is 1.0 to 1 and 0.0 to 0.

In [296]:
data['Medication_given_ALS?'][data['Medication_given_ALS?'] == '1.0'] = '1'
data['Medication_given_ALS?'][data['Medication_given_ALS?'] == '0.0'] = '0'
data['Procedure_given_ALS?'][data['Procedure_given_ALS?'] == '1.0'] = '1'
data['Procedure_given_ALS?'][data['Procedure_given_ALS?'] == '0.0'] = '0'

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3693426213.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Medication_given_ALS?'][data['Medication_given_ALS?'] == '1.0'] = '1'
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3693426213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Medication_given_ALS?'][data['Medication_given_ALS?'] == '0.0'] = '0'
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3693426213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://p

The facility designation where there were multiple types in a single record, were converted to a single value.
This was converted to just one value per record by considering hospital as "general hospital" and others with multiple
specialities given to them as a "multi specialty center".

In [297]:
data['Facility_designation'] = ' '
for i in range(len(data)):
    if data['Facility_Hospital_Designation_List'][i].rstrip() == 'hospital':
        data['Facility_designation'][i] = "hospital"
    elif (data['Facility_Hospital_Designation_List'][i].rstrip().__contains__('hospital') and data['Facility_Hospital_Designation_List'][i].__contains__('pediatric') and len(data['Facility_Hospital_Designation_List'][15327].split(',')) == 2):
        data['Facility_designation'][i] = "trauma center"
    elif (data['Facility_Hospital_Designation_List'][i] == 'none'):
        data['Facility_designation'][i] = 'none'
    else:
        data['Facility_designation'][i] = 'Multi speciality center'

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3644678896.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Facility_designation'][i] = "hospital"
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3644678896.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Facility_designation'][i] = 'Multi speciality center'
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3644678896.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [298]:
data['Facility_designation'].value_counts()

Multi speciality center    99247
hospital                   24062
none                         474
Name: Facility_designation, dtype: int64

Converting the multiple reason for choosing in the "Reason for choosing column" by replacing the multiple reasons with 
one based on the priority that was given in the EMS manual.

The records were considered in this order as below.
1) Patient_choice
2) Closest facility
3) protocol
4) law
5) Medical direction
6) Diversion
7) insurance status

So any record with a value of "Patient choice" in them were automatically considered as "patient choice" alone.

Do no change the order in which this is executed. or the code.

In [299]:
data['cleaned_reason'] = ' '

for i in range(len(data)):
    if (data['ReasonForChoosingHospital'][i].rstrip().__contains__('patient')):
        data['cleaned_reason'][i] = "patient choice"
    elif (data['ReasonForChoosingHospital'][i].rstrip().__contains__('closest')):
        data['cleaned_reason'][i] = "closest facility"
    elif (data['ReasonForChoosingHospital'][i].rstrip().__contains__('protocol')):
        data['cleaned_reason'][i] = "protocol"
    elif (data['ReasonForChoosingHospital'][i].rstrip().__contains__('law')):
        data['cleaned_reason'][i] = 'law choice'
    elif (data['ReasonForChoosingHospital'][i].rstrip().__contains__('medical direction')):
        data['cleaned_reason'][i] = 'on-line/on-scene medical direction'
    elif (data['ReasonForChoosingHospital'][i].rstrip().__contains__('diversion')):
        data['cleaned_reason'][i] = 'diversion'
    elif (data['ReasonForChoosingHospital'][i].rstrip().__contains__('insurance status/requirement')):
        data['cleaned_reason'][i] = 'insurance status/requirement'
    else:
        data['cleaned_reason'][i] = data['ReasonForChoosingHospital'][i]

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/621776197.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_reason'][i] = "closest facility"
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/621776197.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_reason'][i] = "protocol"
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/621776197.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [300]:
data['cleaned_reason'].value_counts()

closest facility                      77747
patient choice                        38025
protocol                               7164
law choice                              232
diversion                               230
insurance status/requirement            172
other                                   159
on-line/on-scene medical direction       54
Name: cleaned_reason, dtype: int64

Cross tabs for checking how the conversion of reason for chossing above have taken place.

In [301]:
pd.crosstab(data['ReasonForChoosingHospital'],data['cleaned_reason'])

cleaned_reason,closest facility,diversion,insurance status/requirement,law choice,on-line/on-scene medical direction,other,patient choice,protocol
ReasonForChoosingHospital,,,,,,,,
closest facility,75504,0,0,0,0,0,0,0
"closest facility,insurance status/requirement,patient choice",0,0,0,0,0,0,1,0
"closest facility,other",1,0,0,0,0,0,0,0
"closest facility,patient choice",0,0,0,0,0,0,6719,0
diversion,0,228,0,0,0,0,0,0
"diversion,closest facility",17,0,0,0,0,0,0,0
"diversion,closest facility,patient choice",0,0,0,0,0,0,3,0
"diversion,on-line/on-scene medical direction",0,0,0,0,1,0,0,0
"diversion,on-line/on-scene medical direction,law enforcement choice",0,0,0,1,0,0,0,0


Cross tab for the facility designation to check how the above conversion took place.

In [302]:
pd.crosstab(data['Facility_designation'],data['Facility_Hospital_Designation_List'])

Facility_Hospital_Designation_List,hospital,"hospital ,pediatric trauma center","hospital ,pediatric trauma center,designated stroke center",none,"pediatric trauma center,level i trauma center,nicu present,burn center","stemi center ,designated stroke center,level ii trauma center,hospital","stemi center ,pediatric trauma center,designated stroke center,hospital","stemi center ,pediatric trauma center,designated stroke center,hospital ,nicu present","stemi center ,pediatric trauma center,designated stroke center,level i trauma center,hospital ,nicu present","stemi center ,pediatric trauma center,designated stroke center,level i trauma center,hospital ,nicu present,burn center"
Facility_designation,,,,,,,,,,
Multi speciality center,0,17285,162,0,66,17514,13478,69,50573,100
hospital,24062,0,0,0,0,0,0,0,0,0
none,0,0,0,474,0,0,0,0,0,0


Code that is does has the conditions in them to choose whether a particular trip related to a patient was 
compliant with the EMS manual norms or not.

It considers the values in the data for "cleaned reason" ,"travel time taken" , "MedicationgivenALS?", "ProceduregivenALS?"
," Facility designation" to give out the classification for a particular record.

Please do not change the order in which this is executed or the control flow of the code below.

In [303]:
def algo(data):
    for i in range(len(data)):
        if (data['cleaned_reason'][i] == 'patient choice'):
            if(data['Travel_time_taken'][i] < 60):
                data['comp'][i] = 'yes'
            else:
                data['comp'][i] = 'no'
                
            
        elif (data['cleaned_reason'][i] == 'closest facility'):
            data['comp'][i] = 'yes'
     
    
        elif ((re.search('protocol', data['cleaned_reason'][i])) or (re.search('insurance status/requirement', data['cleaned_reason'][i]))):
            if (data['clean_primary_impression'][i] == 'burn' or data['clean_primary_impression'][i] == 'cardiovascular - STEMI' or data['clean_primary_impression'][i] == 'stroke' or data['clean_primary_impression'][i] == 'labor' or data['clean_primary_impression'][i] == 'ob' or data['clean_primary_impression'][i] == 'neuro'):
                if ((data['Medication_given_ALS?'][i] == '1') or (data['Procedure_given_ALS?'][i] == '1')):
                    print('ALS')
                    if((data['Facility_designation'][i] == 'hospital') or (data['Facility_designation'][i] == 'none')):
                        data['comp'][i] = 'no'
                    else:
                        data['comp'][i] = 'yes'
                    
                elif ((data['Medication_given_ALS?'][i] == '0') and (data['Procedure_given_ALS?'][i] == '0')):
                    if((data['Facility_designation'][i] == 'hospital') or (re.search('closest', data['cleaned_reason'][i]))):
                        data['comp'][i] = 'yes'
                    else:
                        data['comp'][i] = 'no'
                
                elif ((data['Medication_given_ALS?'][i] != '0') or (data['Medication_given_ALS?'][i] != '1') or (data['Procedure_given_ALS?'][i] != '0') or (data['Procedure_given_ALS?'][i] != '1')):
                    if (data['FinalIncidentType'][i] == 'ALS'):
                        if((data['Facility_designation'][i] != 'hospital') or (data['Facility_designation'][i] != 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                            data['comp'][i] = 'yes'
                        else:
                            data['comp'][i]= 'no'
                    else:
                        if((data['Facility_designation'][i] == 'hospital') or (data['Facility_designation'][i] == 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                            data['comp'][i] = 'yes'
                        else:
                            data['comp'][i]= 'no'
                    
                        
            elif(data['clean_primary_impression'][i] != 'burn' or data['clean_primary_impression'][i] != 'cardiovascular - STEMI' or data['clean_primary_impression'][i] != 'stroke' or data['clean_primary_impression'][i] != 'labor' or data['clean_primary_impression'][i] != 'ob' or data['clean_primary_impression'][i] !='neuro'):
                if ((data['Medication_given_ALS?'][i] == '1') or (data['Procedure_given_ALS?'][i] == '1')):
                    if((data['Facility_designation'][i] != 'hospital') or (data['Facility_designation'][i] != 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                        data['comp'][i] = 'yes'
                    else:
                        data['comp'][i]= 'no'


                elif ((data['Medication_given_ALS?'][i] == '0') and (data['Procedure_given_ALS?'][i] == '0')):
                    if((data['Facility_designation'][i] == 'hospital') or (data['Facility_designation'][i] == 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                        data['comp'][i] = 'yes'
                    else:
                        data['comp'][i] = 'no'
                
                elif ((data['Medication_given_ALS?'][i] != '0') or (data['Medication_given_ALS?'][i] != '1') or (data['Procedure_given_ALS?'][i] != '0') or (data['Procedure_given_ALS?'][i] != '1')):
                    if (data['FinalIncidentType'][i] != 'ALS'):
                        if((data['Facility_designation'][i] == 'hospital') or (data['Facility_designation'][i] == 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                            data['comp'][i] = 'yes'
                        else:
                            data['comp'][i] = 'no'
                    else:
                        if((data['Facility_designation'][i] != 'hospital') or (data['Facility_designation'][i] != 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                            data['comp'][i] = 'yes'
                        else:
                            data['comp'][i] = 'no'
                            
                            
        elif (data['cleaned_reason'][i] != 'protocol' or data['cleaned_reason'][i] != 'closest facility' or data['cleaned_reason'][i] != 'patient choice'):

            if (data['FinalIncidentType'][i] == 'ALS'):
                if((data['Facility_designation'][i] != 'hospital') or (data['Facility_designation'][i] != 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                    data['comp'][i] = 'yes'
                else:
                    data['comp'][i]= 'no'
            else:
                if((data['Facility_designation'][i] == 'hospital') or (data['Facility_designation'][i] == 'none') or (re.search('closest', data['cleaned_reason'][i]))):
                    data['comp'][i] = 'yes'
                else:
                    data['comp'][i] = 'no'
                
#         print(data['comp'][i])

    return data
        

Running the function above for all the records in the data.

In [304]:
data['comp'] = ''

data_algo = algo(data)

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comp'][i] = 'yes'
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comp'][i] = 'yes'
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['co

none_given
none_given
0
0

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comp'][i] = 'yes'
/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comp'][i] = 'yes'



none_given
none_given
none_given
none_given
none_given
there
none_given
0
none_given
none_given
there
0
none_given
none_given
none_given
none_given
none_given
there
none_given
none_given
there

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comp'][i] = 'yes'


...


none_given
there
there
there
there
there
there
there
none_given
0
there
there
there
there
there
there
none_given
there
there
there
there
0
none_given
there
none_given
none_given
none_given
there
there
there
none_given
there
there
there
there
0
none_given
0
0
ALS
none_given
none_given
there
there
0
none_given
there
none_given
none_given
none_given
0
0
0
none_given
none_given
0
none_given
0
none_given
1
ALS
none_given
none_given
none_given
none_given
none_given
0
none_given
none_given
none_given
there
none_given
0
0
none_given
0
none_given
none_given
none_given
0
none_given
none_given
0
none_given
0

/var/folders/hj/thk_8wb95p10ls4prllz9kxr0000gn/T/ipykernel_9772/3242772768.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comp'][i] = 'no'



there
0
none_given
0
0
none_given
there
none_given
none_given
0
0
0
none_given
none_given
none_given
none_given
none_given
none_given
1
ALS
0
none_given
there
0
there
there
0
0
none_given
there
there
none_given
there
0
ALS
none_given
none_given
there
none_given
none_given
there
0
0
0
0
0
0
none_given
0
none_given
none_given
0
none_given
none_given
none_given
none_given
none_given
none_given
there
none_given
there
none_given
none_given
there
0
none_given
none_given
none_given
none_given
none_given
none_given
there
none_given
there
0
none_given
there
none_given
none_given
none_given
there
there
there
0
0
1
ALS
none_given
there
none_given
none_given
0
there
none_given
there
0
none_given
none_given
there
there
there
there
there
none_given
0
none_given
0
there
there
none_given
1
ALS
none_given
none_given
there
there
none_given
there
there
none_given
there
none_given
1
ALS
0
there
none_given
there
there
none_given
0
there
none_given
none_given
there
0
0
there
none_given
ALS
1
ALS
none_given

none_given
0
none_given
none_given
none_given
none_given
0
0
none_given
none_given
none_given
none_given
0
none_given
0
none_given
none_given
0
none_given
none_given
none_given
none_given
none_given
none_given
none_given
none_given
none_given
0
none_given
0
0
none_given
none_given
none_given
none_given
0
none_given
none_given
0
none_given
ALS
none_given
none_given
0
none_given
none_given
none_given
none_given
none_given
0
0
there
none_given
1
ALS
none_given
none_given
none_given
none_given
0
none_given
none_given
none_given
none_given
there
none_given
0
none_given
there
none_given
none_given
none_given
none_given
0
none_given
1
ALS
none_given
none_given
none_given
there
there
0
there
there
there
none_given
none_given
0
none_given
none_given
0
0
ALS
there
none_given
none_given
none_given
none_given
ALS
0
there
none_given
there
none_given
0
none_given
none_given
none_given
none_given
0
none_given
there
0
0
there
0
none_given
0
0
there
none_given
there
there
1
ALS
none_given
1
ALS
there
t

there
0
none_given
0
ALS
none_given
0
none_given
1
ALS
none_given
0
none_given
none_given
none_given
none_given
none_given
0
none_given
there
0
there
none_given
ALS
none_given
there
none_given
there
none_given
none_given
ALS
none_given
0
ALS
there
none_given
there
there
there
1
ALS
0
none_given
there
there
there
there
there
none_given
0
none_given
none_given
0
none_given
there
0
0
ALS
there
none_given
1
ALS
none_given
none_given
0
1
ALS
there
none_given
none_given
there
none_given
none_given
0
none_given
0
there
none_given
there
0
none_given
none_given
0
none_given
there
0
none_given
there
0
none_given
0
none_given
none_given
there
there
there
0
there
there
none_given
none_given
there
none_given
0
none_given
0
there
there
there
none_given
none_given
none_given
ALS
there
there
none_given
none_given
there
0
0
none_given
there
there
there
none_given
0
1
ALS
0
0
ALS
there
none_given
none_given
there
there
there
there
there
none_given
none_given
none_given
none_given
none_given
none_given
n

In [305]:
data_algo['comp'].value_counts()

yes    119355
no       4428
Name: comp, dtype: int64

In [238]:
# data['FinalIncidentType'].value_counts()

Forming different datasets that were used for the visualizations.

Grouping shift day

In [306]:
ow = pd.DataFrame(data_algo.groupby(['ShiftDay','comp'])['PatientID'].count()).reset_index()
ow = ow.rename({'PatientID':'count'}, axis = 1)

In [308]:
uwe = pd.DataFrame(data_algo.groupby(['ShiftDay'])['PatientID'].count()).reset_index()
uwe = uwe.rename({'PatientID':'Total'}, axis = 1)

In [309]:
reu = pd.merge(ow, uwe, on='ShiftDay', how ='inner')

In [310]:
reu['percent'] = (reu['count']/reu['Total'] * 100)
reu
reu.to_csv("/Volumes/Personal/DAEN 690 - Project/groupby_shiftday.csv")

Grouping Tour shift

In [311]:
hd = pd.DataFrame(data_algo.groupby(['TourOfShift','comp'])['PatientID'].count()).reset_index()
hd = hd.rename({'PatientID':'Count'}, axis = 1)

In [312]:
dhf = pd.DataFrame(data_algo.groupby(['TourOfShift'])['PatientID'].count()).reset_index()
dhf = dhf.rename({'PatientID':'Total'}, axis = 1)

In [313]:
hs = pd.merge(dhf, hd, on='TourOfShift', how ='inner')
hs['percent'] = (hs['Count']/hs['Total'] * 100)

In [322]:
hs

,TourOfShift,Total,comp,Count,percent
0,1,41042,no,1397,3.40383
1,1,41042,yes,39645,96.59617
2,2,41426,no,1514,3.65471
3,2,41426,yes,39912,96.34529
4,3,41315,no,1517,3.67179
5,3,41315,yes,39798,96.32821


In [ ]:
hs.to_excel("/Volumes/Personal/DAEN 690 - Project/groupby_tourshift.xlsx")

Group both shift and tour together

In [314]:
oas = pd.DataFrame(data_algo.groupby(['TourOfShift','ShiftDay','comp'])['PatientID'].count()).reset_index()
oas = oas.rename({'PatientID':'count'}, axis =1)

In [315]:
yr = pd.DataFrame(data_algo.groupby(['TourOfShift','ShiftDay'])['PatientID'].count()).reset_index()
yr = yr.rename({'PatientID':'Total'}, axis = 1)

In [316]:
jda = pd.merge(yr, oas, on=['TourOfShift','ShiftDay'] , how ='inner')
jda['percent'] = (jda['count']/jda['Total'] * 100)

In [321]:
jda

,TourOfShift,ShiftDay,Total,comp,count,percent
0,1,A,13729,no,422,3.073785
1,1,A,13729,yes,13307,96.926215
2,1,B,13433,no,467,3.476513
3,1,B,13433,yes,12966,96.523487
4,1,C,13880,no,508,3.659942
5,1,C,13880,yes,13372,96.340058
6,2,A,13501,no,470,3.481224
7,2,A,13501,yes,13031,96.518776
8,2,B,13839,no,514,3.714141
9,2,B,13839,yes,13325,96.285859


In [ ]:
jda.to_excel("/Volumes/Personal/DAEN 690 - Project/groupby_shift_and_tour.xlsx")

Grouping the transportunits with the compliant or not

In [317]:
sub4 = pd.DataFrame(data_algo.groupby(['TransportUnitID','comp'])['PatientID'].count()).reset_index()
sub4 = sub4.rename({'PatientID':'count'},axis = 1)

In [318]:
eui = pd.DataFrame(data_algo.groupby(['TransportUnitID'])['PatientID'].count()).reset_index()
eui = eui.rename({'PatientID':'Total'}, axis = 1)

In [319]:
pqo = pd.merge(eui, sub4, on='TransportUnitID', how='inner')
pqo['percent'] = (pqo['count']/pqo['Total'] *100)

In [320]:
eue = pqo[(pqo['comp'] == 'no') & (pqo['percent'] > 10)]['TransportUnitID']
eue
kkd = pd.merge(eue, pqo, on='TransportUnitID', how='inner')
kkd

,TransportUnitID,Total,comp,count,percent
0,A405E,9,no,1,11.111111
1,A405E,9,yes,8,88.888889
2,A422,16,no,2,12.500000
3,A422,16,yes,14,87.500000
4,A422E,284,no,37,13.028169
5,A422E,284,yes,247,86.971831
6,A438E,40,no,6,15.000000
7,A438E,40,yes,34,85.000000
8,E440,2,no,1,50.000000
9,E440,2,yes,1,50.000000


In [ ]:
kkd.to_excel("/Volumes/Personal/DAEN 690 - Project/groupby_transport.xlsx")

Grouping by Incident First due 

In [323]:
qwe = pd.DataFrame(data_algo.groupby(['IncidentFirstDue','comp'])['PatientID'].count()).reset_index()
qwe = qwe.rename({'PatientID':'count'}, axis =1)

In [324]:
ied = pd.DataFrame(data_algo.groupby('IncidentFirstDue')['PatientID'].count().reset_index())
ied = ied.rename({'PatientID':'Total'}, axis =1)

In [325]:
upqo = pd.merge(ied, qwe, on='IncidentFirstDue', how ='inner')
upqo['percent'] = upqo['count']/upqo['Total']*100

In [326]:
poqw = upqo[(upqo['comp'] == 'no') & (upqo['percent'] > 5)]['IncidentFirstDue']
poqw
kdi = pd.merge(upqo, poqw, on='IncidentFirstDue', how='inner')

In [327]:
kdi

,IncidentFirstDue,Total,comp,count,percent
0,403.0,482,no,25,5.186722
1,403.0,482,yes,457,94.813278
2,405.0,3692,no,270,7.313109
3,405.0,3692,yes,3422,92.686891
4,412.0,638,no,33,5.172414
5,412.0,638,yes,605,94.827586
6,416.0,574,no,54,9.407666
7,416.0,574,yes,520,90.592334
8,417.0,3975,no,285,7.169811
9,417.0,3975,yes,3690,92.830189


In [ ]:
kdi.to_excel("/Volumes/Personal/DAEN 690 - Project/groupby_incident.xlsx")

Average travel time taken for all the transport units

In [330]:
nwks = data_algo.groupby(['TransportUnitID','comp'])['Travel_time_taken'].mean()
eue1 = pqo[(pqo['comp'] == 'no') & (pqo['percent'] > 10)][['TransportUnitID','comp']]
eue1
hsho = pd.merge(eue1,nwks, on=['TransportUnitID','comp'], how ='inner')

In [331]:
hsho

,TransportUnitID,comp,Travel_time_taken
0,A405E,no,23.000000
1,A422,no,17.500000
2,A422E,no,16.108108
3,A438E,no,14.166667
4,E440,no,12.000000
5,M405E,no,12.750000
6,M414B,no,20.000000
7,M414E,no,25.166667
8,M416,no,23.695652
9,M422B,no,12.363636
